# Changing the input current when solving PyBaMM models

This notebook shows you how to change the input current when solving PyBaMM models. It also explains how to load in current data from a file, and how to add a user-defined current function. For more examples of different drive cycles see [here](https://github.com/pybamm-team/PyBaMM/tree/master/results/drive_cycles).

### Table of Contents
1. [Constant current](#constant)
1. [Loading in current data](#data)
1. [Adding your own current function](#function)

## Constant current  <a name="constant"></a>

In this notebook we will use the SPM as the example model, and change the input current from the defualt option. If you are not familiar with running a model in PyBaMM, please see [this](./models/SPM.ipynb) notebook for more details.

In PyBaMM, the current function is set using the parameter "Current function". By defualt this is set to be a constant current provided by the class [`pybamm.GetConstantCurrent`](https://pybamm.readthedocs.io/en/latest/source/parameters/standard_current_functions/get_constant_current.html).  This class takes a single optional argument "current" which is the size of the current in Amperes. If no argument is passed, the value of the current is set by the parameter "Typical current [A]" (see [this](parameter-values.ipynb) notebook for more information about parameters).

In general it is recommended to change the size of a constant current input by changing the parameter "Typical current [A]", since this value is used to non-dimensionlise the model. Below we load the SPM with the defualt parameters, and then change the the typical current 16A. We then explicilty set the current function to be a constant current.

In [1]:
import pybamm
import numpy as np
import os
os.chdir(pybamm.__path__[0]+'/..')

# create the model
model = pybamm.lithium_ion.SPM()

# set the default model geometry
geometry = model.default_geometry

# set the default model parameters
param = model.default_parameter_values

# change the typical current and set a constant discharge using the typical current value
param["Typical current [A]"] = 16
param["Current function"] = pybamm.GetConstantCurrent()


However, you may wish to change the value of the constant current without altering the typical current value used for non-dimensionlidation. For instance, the current function could be change to draw a current of 8A by updating the current function and passing the optional argument "current" to the class `GetConstantCurrent`

In [2]:
# update the value of the current profile *without* changing the typical current used for non-dimensionlisation
param["Current function"] = pybamm.GetConstantCurrent(current=8)

The model may then be processed and solved in the usual way.

In [3]:
# set the parameters for the model and the geometry
param.process_model(model)
param.process_geometry(geometry)

# mesh the domains
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise the model equations
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# Solve the model at the given time points
solver = model.default_solver
n = 100
t_eval = np.linspace(0, 0.2, n)
solution = solver.solve(model, t_eval)

# plot
quick_plot = pybamm.QuickPlot(model, mesh, solution)

import ipywidgets as widgets
widgets.interact(quick_plot.plot, t=widgets.FloatSlider(min=0,max=solution.t[-1],step=0.05,value=0));

interactive(children=(FloatSlider(value=0.0, description='t', max=0.2, step=0.05), Output()), _dom_classes=('w…

## Loading in current data <a name="data"></a>

Data can be loaded in from a csv file using the class [`pybamm.GetCurrentData`](https://pybamm.readthedocs.io/en/latest/source/parameters/standard_current_functions/get_current_data). Data should be given as a function of time (in seconds) and may be dimensional (in Amperes) or non-diemsnional (e.g. C-rate). Optionally, voltage data (in Volts) may be loaded in for convinient plotting and comparison. 

The input csv files should be stored in PyBaMM/input/drive_cycles and have the headings: "time [s]"; either "current [A]" for dimensional data, or "current []" for dimensionless data; and, optionally, "voltage [V]". 

To load in dimensional data we simply provide the filename and set the units to "[A]", e.g.

In [4]:
param["Current function"] = pybamm.GetCurrentData("US06.csv", units="[A]")

For non dimensional data, you also need to provide a current scale. For instance, if the data were C-rate then you would set the current scale to the 1C discharge current. If the 1C discharge current were 24 A and we had some C-rate data in the file car_current.csv, this would be loaded in as

In [5]:
param["Current function"] = pybamm.GetCurrentData("car_current.csv", units="[]", current_scale=24)

As an example, we how to solve the SPM using the US06 drive cycle

In [6]:
model = pybamm.lithium_ion.SPM()

# create geometry
geometry = model.default_geometry

# load parameter values and process model and geometry
param = model.default_parameter_values
param["Current function"] = pybamm.GetCurrentData("US06.csv", units="[A]")
param.process_model(model)
param.process_geometry(geometry)

# set mesh
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise model
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# simulate US06 drive cycle (duration 600 seconds)
tau = param.process_symbol(
    pybamm.standard_parameters_lithium_ion.tau_discharge
).evaluate(0)
t_eval = np.linspace(0, 600 / tau, 600)
solution = solver.solve(model, t_eval)

# plot
quick_plot = pybamm.QuickPlot(model, mesh, solution)

import ipywidgets as widgets
widgets.interact(quick_plot.plot, t=widgets.FloatSlider(min=0,max=solution.t[-1],step=0.001,value=0));

/home/user/Documents/PyBaMM/pybamm/parameters/standard_current_functions/get_current_data.py:85: ModelWarning: Requested time (2387.4040040888344) is outside of the data range [0, 600]
  pybamm.ModelWarning,


interactive(children=(FloatSlider(value=0.0, description='t', max=0.03900922998674932, step=0.001), Output()),…

Note that some solvers try to evaluate the model equations at a very large value of `t` during the first step. This may raise a warning if the time requested by the solver is outside of the range of the data provided. However, this does not affect the solve since this large timestep is rejected by the solver, and a suitable shorter initial step is taken.

## Adding your own current function <a name="function"></a>

A user defined current function can be passed to any model using the class [`pybamm.GetUserCurrent`](https://pybamm.readthedocs.io/en/latest/source/parameters/standard_current_functions/get_user_current). The class takes in a method, which returns the current as a function of time, followed by any keyword arguments required by the method.  

For example, you may want to simulate a sinusoidal current with amplitude A and freqency omega. In order to do so you must first define the method

In [7]:
# create user-defined function
def my_fun(t, A, omega):
    return A * np.sin(2 * np.pi * omega * t)

Note that time *must* me the first arguemnt of the function. The parameters may either be provided as floats, or may be one of the standard parameters provided in PyBaMM (see [here](https://pybamm.readthedocs.io/en/latest/source/parameters)). 

The the model may be loaded and the "Current function" parameter updated to be a `GetUserCurrent` class which calls `my_fun`

In [8]:
model = pybamm.lithium_ion.SPM()

# create geometry
geometry = model.default_geometry

# load default parameter values
param = model.default_parameter_values

# set user defined current function
A = pybamm.electrical_parameters.I_typ
omega = 0.1
param["Current function"] = pybamm.GetUserCurrent(my_fun, A=A, omega=omega)

# process model and geometry
param.process_model(model)
param.process_geometry(geometry)

Note that the parameters in `my_fun` were passed as keyword arguments to the `GetUserCurrent` class. The model may then be solved in the usual way

In [9]:
# set mesh
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise model
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# Example: simulate for 30 seconds
simulation_time = 30  # end time in seconds
tau = param.process_symbol(pybamm.standard_parameters_lithium_ion.tau_discharge).evaluate(0)
npts = 50 * simulation_time * omega  # need enough timesteps to resolve output
t_eval = np.linspace(0, simulation_time / tau, npts)
solution = model.default_solver.solve(model, t_eval)
label = ["Frequency: {} Hz".format(omega)]

# plot current and voltage
output_variables = ["Current [A]", "Terminal voltage [V]"]
quick_plot = pybamm.QuickPlot(model, mesh, solution, output_variables, label)

import ipywidgets as widgets
widgets.interact(quick_plot.plot, t=widgets.FloatSlider(min=0,max=solution.t[-1],step=solution.t[-1]/20,value=0));

/home/user/Documents/PyBaMM/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  if sys.path[0] == '':


interactive(children=(FloatSlider(value=0.0, description='t', max=0.0019504614993374662, step=9.75230749668733…